In [2]:
import requests
import pandas as pd
from pandas import json_normalize
import json
from tqdm import tqdm
from tqdm import trange
from time import sleep

### Testando com o arquivo Geral de Instrumentos

In [3]:
url = 'https://arquivos.b3.com.br/tabelas/table/TradeInformationConsolidated/2022-12-22/1'
r = requests.get(url)
data = json_normalize(json.loads(r.content))
data

,name,friendlyName,friendlyNamePt,friendlyNameEn,columns,values,pageCount
0,TradeInformationConsolidated,Negócios Consolidados do Pregão (Listado),Negócios Consolidados do Pregão (Listado),Consolidated Trades (Listed Segment),"[{'name': 'RptDt', 'type': 4, 'friendlyName': ...","[[2022-12-22T00:00:00, 5GTK11, CASH, 71.42, 76...",2040


In [4]:
page_count = data['pageCount'][0]
page_count

2040

In [5]:
# Convertendo em dataframe
columns =  pd.json_normalize(json.loads(r.content), record_path =['columns'])
columns = columns['friendlyName'].tolist()
columns

['Data',
 'Papel',
 'Segmento',
 'Preço Mínimo',
 'Preço Máximo',
 'Preço Médio',
 'Preço de Fechamento',
 'Oscilação',
 'Ajuste (Preço)',
 'Ajuste (Taxa)',
 'Preço de Referência',
 'Número de Negócios',
 'Quantidade de Contratos Negociados',
 'Volume Financeiro']

#### pegando as primeiras 100 páginas

- Não rodar

In [6]:
all = []

for pagina in trange(1, 201):
#for pagina in trange(1, page_count + 1):
    url = f'https://arquivos.b3.com.br/tabelas/table/TradeInformationConsolidated/2022-12-22/{pagina}'
    r = requests.get(url) #, timeout=5
    if r.headers['content-type'] != 'application/json':
        while r.headers['content-type'] != 'application/json':
            r = requests.get(url)
            if r.headers['content-type'] != 'application/json':
                print(f'Pagina: {pagina} vazia, tentando novamente em 60 segundos')
                sleep(60)



    current = json_normalize(json.loads(r.content), record_path =['values'])
    all.append(current)
    #sleep(0.5)  
        
all
    

100%|██████████| 200/200 [00:30<00:00,  6.60it/s]


[                     0       1     2       3       4       5       6     7   \
 0   2022-12-22T00:00:00  5GTK11  CASH   71.42   76.00   72.50   72.40 -2.16   
 1   2022-12-22T00:00:00  A1AP34  CASH   45.07   45.62   45.52   45.07 -1.20   
 2   2022-12-22T00:00:00  A1BB34  CASH   38.64   39.20   38.88   38.84 -1.72   
 3   2022-12-22T00:00:00  A1CR34  CASH   61.45   62.05   61.68   61.54 -1.75   
 4   2022-12-22T00:00:00  A1DM34  CASH  480.31  480.31  480.31  480.31 -1.78   
 5   2022-12-22T00:00:00  A1IV34  CASH   37.48   42.00   38.08   38.04 -1.75   
 6   2022-12-22T00:00:00  A1LB34  CASH   48.34   50.10   48.75   48.34 -4.48   
 7   2022-12-22T00:00:00  A1LG34  CASH  257.05  257.05  257.05  257.05 -0.75   
 8   2022-12-22T00:00:00  A1LL34  CASH   46.55   49.68   46.68   47.54 -1.99   
 9   2022-12-22T00:00:00  A1LN34  CASH   61.84   61.84   61.84   61.84 -1.93   
 10  2022-12-22T00:00:00  A1MD34  CASH   40.41   43.59   40.72   41.06 -6.10   
 11  2022-12-22T00:00:00  A1ME34  CASH  

In [8]:
df = pd.concat(all)
df.columns = columns
df

,Data,Papel,Segmento,Preço Mínimo,Preço Máximo,Preço Médio,Preço de Fechamento,Oscilação,Ajuste (Preço),Ajuste (Taxa),Preço de Referência,Número de Negócios,Quantidade de Contratos Negociados,Volume Financeiro
0,2022-12-22T00:00:00,5GTK11,CASH,71.42,76.0,72.5,72.4,-2.16,None,None,None,30,347,25160.16
1,2022-12-22T00:00:00,A1AP34,CASH,45.07,45.62,45.52,45.07,-1.2,None,None,None,2,12,546.34
2,2022-12-22T00:00:00,A1BB34,CASH,38.64,39.2,38.88,38.84,-1.72,None,None,None,3,4,155.52
3,2022-12-22T00:00:00,A1CR34,CASH,61.45,62.05,61.68,61.54,-1.75,None,None,None,3,3,185.04
4,2022-12-22T00:00:00,A1DM34,CASH,480.31,480.31,480.31,480.31,-1.78,None,None,None,1,10,4803.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15,2022-12-22T00:00:00,BBDCP224,EQUITY PUT,None,None,None,None,None,None,None,6.65,None,None,None
16,2022-12-22T00:00:00,BBDCP227,EQUITY PUT,None,None,None,None,None,None,None,5.45,None,None,None
17,2022-12-22T00:00:00,BBDCP229,EQUITY PUT,None,None,None,None,None,None,None,7.12,None,None,None
18,2022-12-22T00:00:00,BBDCP233,EQUITY PUT,None,None,None,None,None,None,None,7.83,None,None,None


### Pegando instrumentos do cadastro, mas já filtrados - teste com DI1

- Dessa forma não preciso pegar mais de 2500 páginas, mas sim apenas as que interessam

In [9]:
import requests
import pandas as pd
from pandas import json_normalize
import json
from tqdm import tqdm
from tqdm import trange
from time import sleep

In [14]:
url = 'https://arquivos.b3.com.br/tabelas/table/InstrumentsConsolidated/2022-11-25/1?filter=DI1'
r = requests.get(url, timeout=10)
data = json_normalize(json.loads(r.content))
data

ReadTimeout: HTTPSConnectionPool(host='arquivos.b3.com.br', port=443): Read timed out. (read timeout=10)

In [ ]:
page_count = data['pageCount'][0]
page_count

3

In [ ]:
# Convertendo em dataframe
columns =  pd.json_normalize(json.loads(r.content), record_path =['columns'])
columns = columns['friendlyName'].tolist()
columns

['Data',
 'Papel',
 'Nome do Segmento',
 'Nome do Mercado',
 'Notional do Futuro',
 'Tamanho do Lote Padrão',
 'Nome da Instituição',
 'Ativo Objeto',
 'Data de Expiração',
 'Preço de Exercício']

In [13]:
all = []

for pagina in trange(page_count+1):
    url = f'https://arquivos.b3.com.br/tabelas/table/InstrumentsConsolidated/2022-12-22/{pagina}?filter=DI1'
    r = requests.get(url) #, timeout=5
    current = json_normalize(json.loads(r.content), record_path =['values'])
    all.append(current)
    #sleep(0.5)  
        
di = pd.concat(all, ignore_index=True)
di. columns = columns
di

  0%|          | 0/4 [00:05<?, ?it/s]


ReadTimeout: HTTPSConnectionPool(host='arquivos.b3.com.br', port=443): Read timed out. (read timeout=5)

## Pegando dolar, di e indice de uma vez

- Função final


In [11]:
import requests
import pandas as pd
from pandas import json_normalize
import json
from tqdm import trange
from tqdm import tqdm
from time import sleep
from datetime import datetime
import time

In [12]:
# Se entrar no modo de espera gravar log com o horário exato
# comitar código

now = datetime.now().strftime('%H:%M:%S')
now

def espera_request(url, wait=300):
    '''
    Função que faz requisição e caso não seja bem sucedida (pois a b3 bloqueia caso passe de um limite),
    espera o tempo pré-definido e tenta novamente até que seja bem sucedida
    param url: url da requisição
    return: resposta da requisição
    '''
    while(True):
        
        r = requests.get(url)
        if r.headers["content-type"] == 'application/json':
             break
        else:
            print(f'Erro na requisição às {now}, esperar {wait//60} min e tentar novamente')
            sleep(wait)
            print(f'Fazendo requisição para {url} às {now}')
    
    return r


def get_instruments(date):
    '''
    Função para obter os instrumentos de uma data específica
    param date: data no formato yyyy-mm-dd
    return: dataframe com os instrumentos
    '''
    all = []
    assets = ['DI1', 'IND', 'DOL']

    for asset in tqdm(assets):

        base_url = f'https://arquivos.b3.com.br/tabelas/table/InstrumentsConsolidated/{date}/1?filter={asset}'
        r = espera_request(base_url, 1800)
        page_count = json_normalize(json.loads(r.content))['pageCount'][0]
        columns =  json_normalize(json.loads(r.content), record_path =['columns'])['name'].tolist()

        for pagina in trange(1,page_count+1):
            
            url = f'https://arquivos.b3.com.br/tabelas/table/InstrumentsConsolidated/{date}/{pagina}?filter={asset}'
            r = espera_request(url, 1800)
            all.append(json_normalize(json.loads(r.content), record_path =['values']))

        print(f'Fim da requisição para {asset}')
    
    df = pd.concat(all, ignore_index=True)
    df.columns = columns
            
    return df

In [13]:
# coding efficiency
start_time = time.time()

instrumentos_b3 = get_instruments('2022-11-24')

end_time = time.time()
print(f"Result calculated in {(end_time - start_time)//60} minutes and {(end_time - start_time)%60} seconds")

 33%|███▎      | 1/3 [00:01<00:03,  1.69s/it]

Fim da requisição para DI1


 67%|██████▋   | 2/3 [00:02<00:01,  1.03s/it]

Fim da requisição para IND


100%|██████████| 3/3 [00:28<00:00,  9.44s/it]

Fim da requisição para DOL
Result calculated in 0.0 minutes and 28.385669231414795 seconds


In [18]:
#transfomando data - Agregar na função se for necessário
instrumentos_b3['RptDt'] = pd.to_datetime(instrumentos_b3['RptDt'], format='%Y-%m-%d')#.strftime('%Y/%m/%d')
instrumentos_b3['XprtnDt'] = pd.to_datetime(instrumentos_b3['XprtnDt'], format='%Y-%m-%d')
instrumentos_b3

,RptDt,TckrSymb,SgmtNm,MktNm,AsstQtnQty,AllcnRndLot,CrpnNm,Asst,XprtnDt,ExrcPric
0,2022-11-24,DCDI11F,ODD LOT,EQUITY-CASH,NaN,1,FIDC DE TRANSMISSÃO INFINITY DI,DCDI,NaT,None
1,2022-11-24,DI1F23,FINANCIAL,FUTURE,1.0,1,None,DI1,2023-01-02,None
2,2022-11-24,DI1F24,FINANCIAL,FUTURE,1.0,1,None,DI1,2024-01-02,None
3,2022-11-24,DI1F25,FINANCIAL,FUTURE,1.0,1,None,DI1,2025-01-02,None
4,2022-11-24,DI1F26,FINANCIAL,FUTURE,1.0,1,None,DI1,2026-01-02,None
...,...,...,...,...,...,...,...,...,...,...
1958,2022-11-24,DOLZ22P007300,FINANCIAL,OPTIONS ON SPOT,1000,1,None,DOL,2022-12-01,7300
1959,2022-11-24,DOLZ22P007400,FINANCIAL,OPTIONS ON SPOT,1000,1,None,DOL,2022-12-01,7400
1960,2022-11-24,DOLZ22P007500,FINANCIAL,OPTIONS ON SPOT,1000,1,None,DOL,2022-12-01,7500
1961,2022-11-24,DOLZ22P007600,FINANCIAL,OPTIONS ON SPOT,1000,1,None,DOL,2022-12-01,7600


In [19]:
instrumentos_b3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1963 entries, 0 to 1962
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   RptDt        1963 non-null   datetime64[ns]
 1   TckrSymb     1963 non-null   object        
 2   SgmtNm       1963 non-null   object        
 3   MktNm        1963 non-null   object        
 4   AsstQtnQty   1953 non-null   object        
 5   AllcnRndLot  1963 non-null   int64         
 6   CrpnNm       10 non-null     object        
 7   Asst         1963 non-null   object        
 8   XprtnDt      1953 non-null   datetime64[ns]
 9   ExrcPric     1882 non-null   object        
dtypes: datetime64[ns](2), int64(1), object(7)
memory usage: 153.5+ KB


In [20]:
last_bizday = "2022-11-24"
instrumentos_b3 = instrumentos_b3[(instrumentos_b3['MktNm'] == 'FUTURE') & (instrumentos_b3['XprtnDt'] >= last_bizday)]

# Pegando os contratos de índices futuros mais próximos do vencimento
di1 = instrumentos_b3[(instrumentos_b3['TckrSymb'].str.contains('DI1', case=False))].sort_values(by='XprtnDt')['TckrSymb'].values[0]
dol = instrumentos_b3[(instrumentos_b3['TckrSymb'].str.contains('DOL', case=False))].sort_values(by='XprtnDt')['TckrSymb'].values[0]
ind = instrumentos_b3[(instrumentos_b3['TckrSymb'].str.contains('IND', case=False))].sort_values(by='XprtnDt')['TckrSymb'].values[0]
print(f'di1: {di1}')
print(f'dol: {dol}')
print(f'ind: {ind}')



di1: DI1Z22
dol: DOLZ22
ind: INDZ22
